## publishing context

This article addresses publishing context.  The workflow scenario is authoring documentation in a Jupyter notebook.  The requirement is to support rmarkdown html, rmarkdown pdf, and pkgdown article output.  Details of these document pipelines are in the appendix.

### jupyter header yaml

It can be convenient to have a copy and paste YAML header when creating new jupyter documents.  

```yaml
---
title: "authoring"

all_doc: &shared
  pandoc_args: 
    - !with_env "--resource-path={R_HERE_HERE}/vignettes"

output:
  html_document:
    <<: *shared
    theme: cerulean
    highlight: espresso
  pdf_document:
    <<: *shared
---
```

Above, we show the defaults used by metayer. However, these are easily changed via the config.yml file.  Documentation is in the [configuration vignette](./metayer-config.html)

## with_pander

`pander` converts to markdown.  `with_pander` adds logging and context dependent output.

In [ ]:
# convert to markdown, with a message
pander({
  message("hello, stderr")
  pcd
})

In [ ]:
# publishing context upgrade!
with_pander({
  message("hello, stderr.  hello, log.")
  pcd
})

`with_pander` can be used with plots.  It does this by generates an image file and injecting a markdown link.  By default the image format is PNG, which is fine for HTML, but isn't ideal for PDF output.

In [ ]:
# using with_pander to generate a PNG file
with_pander({
  plot(pcd$x, pcd$y)
})

However, a screen grab of PDF output illustrates the image scaling artifacts that arise with a raster image.

![](figure/pdf-raster.png)

## sure_plot

`sure_plot` addresses this concern by inspecting publishing context.  This ensures that an HTML document will be paired with png raster images whereas a PDF document will be paired with svg vector images.

In [ ]:
# Produce a context aware plot
sure_plot(
  {
    plot(pcd$x, pcd$y)
  },
  height = 5,
  width = 5,
  res = 96,
  par_opts = list(bg = "#f1d08f")
)

And the corresponding screen grab from the PDF.

![](figure/pdf-vector.png)


## appendix

### publishing formats

This snippet builds all three output formats:

```r
article_name <- "authoring"

pub_ipynb_to_rmd(
  "./vignettes/authoring.ipynb",
  "./vignettes/authoring.Rmd"    
)

# create standalone html and pdf
rmarkdown::render(
  input = "./vignettes/authoring.Rmd",
  output_format = "all"
)

# build the pkgdown article
pkgdown::build_article("authoring")

```

### incremental results

Using knitr, we can convert from the Rmarkdown format to a regular markdown file. This would be suitable for direct pandoc processing.

```r
# be intentional about the image format
withr::with_options(
  list(knitr.chunk.dev = "jpeg"),
  {
    knitr::knit(
      "./vignettes/authoring.Rmd",
      "./vignettes/authoring_jpeg.md"
    )
  }
)
```
